```{contents}
```
## Conditional Edge

A **Conditional Edge** in LangGraph is a **control-flow mechanism** that dynamically selects the next node during execution based on the **current state**.
It allows LangGraph to implement **branching logic, decision making, routing, loops, and autonomous behavior**.

---

### **1. Motivation**

Static pipelines cannot adapt at runtime.
LLM systems require **dynamic decisions**:

| Requirement     | Example                              |
| --------------- | ------------------------------------ |
| Tool selection  | Choose between search or calculation |
| Loop control    | Continue reasoning or stop           |
| Error recovery  | Retry or fallback                    |
| Planning        | Re-route based on partial results    |
| Human oversight | Request approval if risk is high     |

Conditional edges provide this adaptivity.

---

### **2. Formal Definition**

A conditional edge is a function:

[
f: \text{State} \rightarrow \text{Next Node Key}
]

It maps the **current state** to the identifier of the **next node**.

---

### **3. Core API Pattern**

```python
builder.add_conditional_edges(
    source_node,
    condition_function,
    {
        "route_key_1": "node_a",
        "route_key_2": "node_b",
        END: END
    }
)
```

* `condition_function` returns one of the **route keys**
* The mapping defines the actual graph transitions

---

### **4. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    value: int

def router(state: State):
    if state["value"] > 10:
        return "large"
    return "small"

def large_node(state): return {"value": state["value"] - 10}
def small_node(state): return {"value": state["value"] + 1}

builder = StateGraph(State)

builder.add_node("large", large_node)
builder.add_node("small", small_node)

builder.set_entry_point("small")

builder.add_conditional_edges(
    "small",
    router,
    {
        "large": "large",
        "small": "small",
        END: END
    }
)

graph = builder.compile()
```

---

### **5. How Conditional Edges Execute**

1. Node executes and updates state
2. Conditional function evaluates updated state
3. Selected route determines next node
4. Execution continues until `END`

This implements a **runtime decision system**.

---

### **6. Advanced Usage Patterns**

#### **A. Loop Control**

```python
def should_continue(state):
    return END if state["done"] else "repeat"
```

#### **B. Tool Routing**

```python
def choose_tool(state):
    return "search" if "?" in state["query"] else "calculate"
```

#### **C. Error Recovery**

```python
def handle_error(state):
    return "retry" if state["error"] else "success"
```

#### **D. Human Approval**

```python
def approval_gate(state):
    return "human" if state["risk"] > 0.8 else "auto"
```

---

### **7. Conditional Edge vs Regular Edge**

| Feature        | Regular Edge | Conditional Edge |
| -------------- | ------------ | ---------------- |
| Decision       | Static       | Dynamic          |
| Based on State | No           | Yes              |
| Supports Loops | Limited      | Fully            |
| Enables Agents | No           | Yes              |

---

### **8. Production Considerations**

| Concern         | Solution            |
| --------------- | ------------------- |
| Infinite loops  | Step limit, timeout |
| Safety          | Human-in-loop       |
| Debugging       | State tracing       |
| Fault tolerance | Retry edges         |
| Explainability  | Log route decisions |

---

### **9. Conceptual Interpretation**

Conditional edges convert a static graph into a **state-driven decision machine**:

> **State → Evaluate → Route → Execute → Update State**

This is the core mechanism that enables **agents, planners, self-correcting systems, and autonomous workflows** in LangGraph.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    value: int

def increment(state: State):
    new = state["value"] + 1
    print("Increment →", new)
    return {"value": new}

def halve(state: State):
    new = state["value"] // 2
    print("Halve →", new)
    return {"value": new}

from langgraph.graph import END

def router(state: State):
    if state["value"] == 7:
        return END
    elif state["value"] < 10:
        return "inc"
    else:
        return "half"


In [2]:
from langgraph.graph import StateGraph

builder = StateGraph(State)

builder.add_node("inc", increment)
builder.add_node("half", halve)

builder.set_entry_point("inc")

builder.add_conditional_edges(
    "inc",
    router,
    {
        "inc": "inc",
        "half": "half",
        END: END
    }
)

builder.add_conditional_edges(
    "half",
    router,
    {
        "inc": "inc",
        "half": "half",
        END: END
    }
)

graph = builder.compile()


In [3]:
result = graph.invoke({"value": 3})
print("\nFinal State:", result)


Increment → 4
Increment → 5
Increment → 6
Increment → 7

Final State: {'value': 7}
